In [41]:
from typing import List, Dict
from sagemaker import Session
import boto3
import json

# Building Foundation Model based Applications

Working with LLMs to provide you with advanced reasoning and routing capabilities is easy to get started with. After all the models understand human level instructions, and provide formattable string outputs as a result. 

Yet, when you are looking to develop production ready applications you will require robust data integrations to provide input to your model, you want to solve the alignment problem with LLMs, tune the behavior to your specific corporate governance and brand messaging.

Complex workflows will involve multiple stages to create intermediate results. These stages require the model to switch roles, or might involve optimized task models to be more efficient, or even fine-tuned further on the specific tasks. 

All of this creates complexity when creating and maintaining your LLM based applications in practice.



In [48]:
#load stored variables from previous notebook
%store -r

# Initialize key environment variables
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sm_client = boto3.client("sagemaker", aws_region)
model_version = "*"

print(inference_model)

raglc-huggingface-textgeneration1-gpt-j-2023-06-21-09-49-17-052


In [43]:
!pip install -r requirements.txt

zsh:1: /Users/xfrankx/Documents/Coding/ml/generativeai/gai-bootcamp/Day1/.venv/bin/pip: bad interpreter: /Users/xfrankx/Documents/Coding/generativeai/gai-bootcamp/Day1/.venv/bin/python3.11: no such file or directory
  Using cached anyio-3.7.0-py3-none-any.whl (80 kB)
  Using cached appnope-0.1.3-py2.py3-none-any.whl (4.4 kB)
  Using cached argon2_cffi-21.3.0-py3-none-any.whl (14 kB)
  Using cached argon2_cffi_bindings-21.2.0-cp38-abi3-macosx_10_9_universal2.whl (53 kB)
  Using cached arrow-1.2.3-py3-none-any.whl (66 kB)
  Using cached asttokens-2.2.1-py2.py3-none-any.whl (26 kB)
  Using cached backcall-0.2.0-py2.py3-none-any.whl (11 kB)
  Using cached bleach-6.0.0-py3-none-any.whl (162 kB)
  Using cached boto3-1.26.153-py3-none-any.whl (135 kB)
  Using cached botocore-1.29.153-py3-none-any.whl (10.9 MB)
  Using cached cffi-1.15.1-cp311-cp311-macosx_11_0_arm64.whl (174 kB)
  Using cached comm-0.1.3-py3-none-any.whl (6.6 kB)
  Using cached debugpy-1.6.7-py2.py3-none-any.whl (4.9 MB)
  Us

## Using the power of LangChain
Recently the community unified their efforts on a high-level Framework to ease the development of foundation model based applications.
LangChain was developed to ease the integration of models deployed, or used over proprietary APIs. It lets you easily integrate models into your application, manage the templates for prompts to tune your model behaviour, provide IO, add memory and chain multiple reasoning and action steps. 

### What is LangChain
LangChain is a framework for developing applications powered by language models.

It helps us with:
1. **Integration** - Bring external data, such files, databases, webcontent, API data to your application
2. **Coordination** - Develop reusable, modularized pipelines to execute complex workflows 
3. **Agency** - Enable your LLM to interact with it's environmetn via decision making

## Benefits of using the Framework
1. Components - LangChain makes it easy to swap out abstractions and components necessary to work with language models.

2. Customized Chains - LangChain provides out of the box support for using and customizing 'chains' - a series of actions strung together.

3. Speed 🚢 - This team ships insanely fast. You'll be up to date with the latest LLM features.

4. Community 👥 - Wonderful discord and community support, meet ups, hackathons, etc.


## Connecting your model on AWS
To work with your models on AWS you can use either an integration with the SageMaker endpoint, or in the future directly talk to the Bedrock API. 

For now, let's look at how to work with a custom SageMaker Model Endpoint.

In [60]:
import json
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint

parameters = {
    "max_length": 300,
    "num_return_sequences": 1,
    "top_k": 30,
    "top_p": 0.50,
    "do_sample": True,
    "temperature": 0.8
}


class ContentHandler(LLMContentHandler):
    content_type = "application/x-text"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        print(prompt)
        print(type(prompt))
        input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["generated_texts"][0]


content_handler = ContentHandler()

sm_llm = SagemakerEndpoint(
    endpoint_name=inference_model,
    region_name=aws_region,
    model_kwargs=parameters,
    content_handler=content_handler,
)

print(f"Loaded model endpoint: {inference_model}")

Loaded model endpoint: raglc-huggingface-textgeneration1-gpt-j-2023-06-21-09-49-17-052


In [61]:
sm_llm("What day comes after Friday")

What day comes after Friday
<class 'str'>


TypeError: list indices must be integers or slices, not str

# Creating a basic langchain application

Every LangChain application centers around your LLM model. This can be either a deployed inference endpoint, or a managed service (Bedrock, OpenAI API). The framework provides a series of out of the box integrations in the `llms` module and can be easily expanded to your use case. 



## Models

A model takes a series of messages and returns a message as output

You can choose between:
1. **LanguageModel** Takes text and returns text
2. **Chat Model** Takes a series of messages and returns a message output
3. **Embedding Models** Transform your text into a latent space vector to power similarity search

### Language Models
A wrapper around a typical text input, text output interaction with the model. No structure is expected, and no structure is maintained. Good starting point for many non-chat applications. 

In [ ]:
#TODO: Utilize the local endpoint deployed
llm = OpenAI(openai_api_key=vars['OPENAI_API_KEY'])

Now that we established our connection, we can query the model by sending it instructions as text.

In [ ]:
text = "What would be a good name for a template factory library for prompt engineering"
print(llm(text))

In [ ]:
complex_prompt = """
You are working to extract information from text. You are waiting for the input without asking questions. 
You are not responding to any questions asked directly. 

Input: During last years primary elections, Senator Charles Ray was able to increase voter share in Wisconsin
from 12.5% to 15.5%.
Output: [("Charles Ray", "Position", "Senator"), (")]
"""

In [ ]:
print(llm(complex_prompt))

And we can use vanilla string formatting to integrate information into our models. This allows us to pass information in a structed manner into the model, masking the general nature of the model. This allows to create all the common products you see being built natively on LLMs. 

In [ ]:
architect_prompt = """
Act as a solution architect experienced on AWS. You are analysing customer requirements to create
well-architected solution architectures that you present to the customer. You are detailled, kind and
focussed. You are not engaging in general discussions outside of the scope of system design. 

#System Requirements:
{requirements}
#Scale:
{scale}
#Features:
{features}

Given the requirements the recommended architecture for a solution on AWS.
A:
"""

In [ ]:
prompt = architect_prompt.format(
    requirements="test", 
    scale="testing", 
    features="Auch was"
)
print(prompt)

In [ ]:
print(llm(prompt))

This works but can get a bit clunky when you try to scale it out to more complex use cases. The next type of model wrapper provides a solution to this problem

### Chat Models
These models structure their input and outputs with Schemata that enable you to reason about the expected input and output process. This helps to build more complex designs by seperating the inputs used to provide the model with its role instruction, the query and the context to the query. 

In [ ]:
from langchain.chat_models import ChatAnthropic
chat_llm = ChatAnthropic(model="claude-v1", temperature=0.7, anthropic_api_key=vars['ANTROPIC_API_KEY'])

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
chat_llm([
    SystemMessage(content="You are an unhelpful AI bot that makes jokes at whatever the user says."),
    HumanMessage(content="I would like to go to New York, how should i do this?"),
    AIMessage(content="???")
])

## Schemata

We see that ChatModels use typed classes to structure inputs. This is an example of a LangChain `Schema`, but its just one of many. 

LangChain currently provides the following schemata:

* **Text** The primary interface to interact with a model (used with LanguageModels
* **ChatMessages** What you saw we defined up with the ChatModel
* **Examples** Input/output pairs acting as context for fine tuning model behavior in n-shot learning
* **Document** Piece of unstructured data holding data as content and metadata for retrieval in context

### ChatMessages Schema
The primary interface through which end users interact with these is a chat interface. For this reason, some model providers even started providing access to the underlying API in a way that expects chat messages.

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

hum_msg = HumanMessage(content='inputs send to the model by the user', additional_kwargs={}, example=True)
hum_msg

In [ ]:
sys_msg = SystemMessage(content='Instructions to the model', additional_kwargs={})
sys_msg

In [ ]:
ai_msg = AIMessage(content='Context answer providing further input to the model', additional_kwargs={})
ai_msg

This structure allows us to simply pass multiple requests into a model for batch processing, making application integration easier

In [ ]:
# Generate completions for multiple sets of messages
batch_messages = [
    [   SystemMessage(content="You are a helpful assistant that translates English to German."),
        HumanMessage(content="What a wonderful day we had at the beach this late summer.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to German."),
        HumanMessage(content="What a wonderful day we had at the beach this late summer.")
    ]
]

###  Examples
These can be inputs/outputs for a model or for a chain. Both types of examples serve a different purpose. Examples for a model can be used to finetune a model. Examples for a chain can be used to evaluate the end-to-end chain, or maybe even train a model to replace that whole chain.

In [ ]:
#TODO: Add code on how to use the examples

###  Documents
A piece of unstructured data. Consists of page_content (the content of the data) and metadata (auxiliary pieces of information describing attributes of the data).

In [ ]:
#TODO: Add code on how to use Documents 

## EXERCISE 1
We are working to enable our marketing team to provide customized sales emails at scale. You are asked to create to engineer a prompt for a custom marketing email copy creation pipeline. 

You will be given the following inputs that are collected on the users in your database:
* Name 
* Age
* Interest (List of strings)

You will also be given a recommended product to personalize-recommend to the user
* Product described as a dictionary of attributes (document from DB)


Work to complete the function below:

In [ ]:
from typing import List
#TODO Rewrite for callcenter use case

# Complete the function 
def create_email_copy(name: str, age: int, interests: List[str], product: dict) -> str:
    """
    The email should be personalized, be age appropriate, target the interests 
    of the person and market the product you are selling. 

    Fill in this template using string formatting and a combination of the prompt
    engineering techniques you have learned previously. 
    """
    pass

In [ ]:
# Define the product you are selling. Play with the level of detail

product = {}

In [ ]:
# Define a set of users to generate eamils for
users = [
    {
    "name": "",
    "age": 0,
    "intesrests": [],
    "product": product
    }
]

In [ ]:
# Test your marketing output
for user in users:
    print("\n\n")
    print(llm(create_email_copy(user)))


# Prompt templates 

When building more complex scenarios, managing the parameters placed into the templates can be too complex for simple string injection methods. Eventually you want to describe the interface in a more programmatic way. Here the `PromptTemplate` helps to define verified input variables to be utilized in the format string.

### The PromptTemplate class 

Let's structure our architecture template to make it reusable in our architecture.

In [ ]:
from langchain.prompts import PromptTemplate

# First we can define an exposed parameter interface to the format string
prompt = PromptTemplate(
    input_variables=["requirements", "scale", "features"],
    template=architect_prompt,
)

The template can be asked to format itself, returning the compiled format string for review.

In [ ]:
final_prompt = architect_prompt.format(
    requirements="External facing web application written in Javascript, global deployment",
    scale="Average of 500 requests per minute, scale events up to 3000 requests per second",
    features="Mobile website, desktop version, javascript"
)

In [ ]:
llm(final_prompt)

Having a string output is nice and dandy, but what if we want to create structure returns for further use in our applications. For example, how would we continue working with an extracted set of attributes from a text in a parser scenario? 

Is a string good enough, or would we rather want to return a named tuple, dict or list of class instances from the model?

In [ ]:
references = PromptTemplate(
    input_variables=['text'],
    template="""
    You identify named entities in the text and extract relations amongst them. 
    You do not answer questions, and you do not ask questions.
    It is very important to extract all references you find. Do not skip any in your output.

    # Examples:
    The Dow Jones closed with a plus of 1456 points // ("Dow Jones", "closed", "1456 points")

    Q: {text} // 
    """
)

print(references("Mister Higgins bought the old house next to the woods."))

In [ ]:
#TODO: Implement the ParserClass
class ReferenceOutputParser(BaseOutputParser):
    pass

# Langchaining 

More complex workflows will involve the abbility of the model to react to specific subsets of tasks with specialized sequence of behaviors. We would capture these subsections of our application into modules called `chains`. 

Each chain structures the interaction with a model through specialized prompts, optional examples and optional output parsers. 

The langchain chains module contains the classes to help us easily create specialized sequences of chains that can receive inputs from previous model inferences as structured outputs. 

### Basic LLMChain



In [ ]:
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["product"],
    template="""
    Create a marketable company name for a company selling {product}
    """
)
chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
chain.run("colorful socks")

Answering multiple questions at a time

In [ ]:
qs = [
    {'product': "Kids kites"},
    {'product': "Running shoes"},
    {'product': "Tennis sports wear"},
]

res = chain.generate(qs)
res

Let's consider a more realistic problem to solve with LLMs that we can not easily solve with a simngle prompt. Here we see the power of chaining a series of steps conducted by an agent using serialized IO of the results.

In [ ]:
services_prompt = PromptTemplate(
    input_variables=["product"],
    template="""
    """,

)

In [ ]:
#TODO: First create name of the company, then provide a slogan, then create a mission and vision statement

slogan_prompt = PromptTemplate(
    input_variables=['product', 'company_name'],
    template="""
    You are desigining a corporate identity for {company_name}.
    The company is selling {product} and wants to be 
    """
)
# Slogan chain
slogan_chain = LLMChain(
    llm=llm,
    prompt=slogan_prompt
)

# EXERCISE 2
Let's bring it all together to create a custom LangChain that can build up a cooking article for our new online magazine. 

We will develop a series of chains to expand on a set of initial travel destinations to cover. 

Each article will contain:
* A overview section of the travel destination
* A list of things to do in the region
* A recipe for a famous local dish (description, list of ingredients, step by step instruction)

### Target
Your system should create the articles based on the following inputs:

* Travel destination (City, Country)

In [ ]:
# Let's start by creating the subchains

# 1. Create the destination_overview chain


# 2. Create the recommended_activities chain


# 3. Create the famous_local_dish recommender 


Each of the chains should be constructed for a subset of the workflow that is self contained. Extending too many tasks at once can overload the model and lead to poor results. 

In [4]:
#  Now unifiy the elements and ensure the information flow properly

In [ ]:
# Create a list of locations to cover in your reports
location_list = []

In [ ]:
# Execute the chain against the location_list
result = ...

## Summary
We have seen how LangChain provides us with a series of abstractions on the core building blocks of LLM based applications. 

We have learned to connect our models, create custom Templates, use Schemata to structure our message flow, chain a series of steps and structure the models outputs to further pass into downstream systems. 

Each component of the system is under continued development and likely you will see the library change as it continues to mature. 

Still, we believe LangChain to be a useful abstraction to develop faster and build a more sustainable code base. 